In [ ]:
import numpy as np
import sympy
import math
import matplotlib.pyplot as plt
from pyodesys.symbolic import SymbolicSys
sympy.init_printing()
%matplotlib inline

In [ ]:
def rhs(t, y, p, backend=math):
    return [y[1], backend.sin(t)-p[0]*y[0]]

In [ ]:
odesys = SymbolicSys.from_callback(rhs, 2, 1)

In [ ]:
odesys.exprs

In [ ]:
def analytic(tout, init_y, p):
    t, (y0, y1), (k,) = odesys.indep, odesys.dep, odesys.params
    c1, c2 = sympy.symbols('c1 c2')
    sqk = sympy.sqrt(k)
    f = c1*sympy.cos(sqk*t) + c2*sympy.sin(sqk*t) + sympy.sin(t)/(k-1)
    dfdt = f.diff(t)
    t0 = tout[0]
    sol, = sympy.solve([f.subs(t, t0) - init_y[0],
                       dfdt.subs(t, t0) - init_y[1]],
                      [c1, c2], dict=True)
    sol[k] = p[0]
    exprs = [f.subs(sol), dfdt.subs(sol)]
    cb = sympy.lambdify([t], exprs)
    return np.array(cb(tout)).T

In [ ]:
def integrate_and_plot(system):
    init_y = [0, 0]
    p = [2]
    result = system.integrate([0, 80], init_y, p, integrator='cvode', nsteps=5000)
    fig, axes = plt.subplots(1, 2, figsize=(10, 4))
    result.plot(ax=axes[0])
    yref = analytic(result.xout, init_y, p)
    result.plot(y=result.yout - yref, ax=axes[1])
    print({k: v for k, v in result.info.items() if not k.startswith('internal')})

In [ ]:
asys = odesys.as_autonomous()

In [ ]:
integrate_and_plot(odesys)

In [ ]:
integrate_and_plot(asys)

In [ ]:
odesys.get_jac()

In [ ]:
asys.get_jac()